# Data EDA & Extension to 1000 Rows


In [5]:
import pandas as pd
from pathlib import Path

# Directorio base correcto
base_dir = Path("data")

# Archivos esperados
files = {
    "affiliates": base_dir / "affiliates.csv",
    "players": base_dir / "players.csv",
    "transactions": base_dir / "transactions.csv"
}

# Verificar existencia
for name, path in files.items():
    if not path.exists():
        raise FileNotFoundError(f"No se encontró el archivo: {path.resolve()}")

# Leer archivos
affiliates = pd.read_csv(files["affiliates"])
players = pd.read_csv(files["players"])
transactions = pd.read_csv(files["transactions"])

print("Shapes:", len(affiliates), len(players), len(transactions))
display(
    affiliates.head(3),
    players.head(3),
    transactions.head(3)
)


Shapes: 10 11 10


,id,code,origin,redeemed_at
0,1,DWEHSP,YouTube,2024-01-10 17:07:01+00:00
1,2,QGKTBC,Discord,2024-02-13 05:40:50+00:00
2,3,YRLUJA,X,2024-02-24 04:35:15+00:00


,id,affiliate_id,country_code,is_kyc_approved,created_at,updated_at
0,1,1.0,DE,True,2024-01-10 17:07:01+00:00,2024-02-08 09:05:26+00:00
1,2,NaN,BR,True,2024-02-09 19:19:27+00:00,2024-03-05 08:06:20+00:00
2,3,2.0,BR,True,2024-02-13 05:40:50+00:00,2024-02-17 10:58:17+00:00


,id,timestamp,player_id,type,amount
0,1,2024-01-10 17:07:01+00:00,7,Withdraw,213.13
1,2,2024-02-09 19:19:27+00:00,4,Withdraw,182.64
2,3,2024-02-13 05:40:50+00:00,8,Withdraw,184.22


In [6]:
# Helper functions for validation
import pandas as pd
import numpy as np

def find_col(df, candidates):
    cols_lc = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand in cols_lc:
            return cols_lc[cand]
    for c in df.columns:
        cl = c.lower()
        for cand in candidates:
            if cand in cl:
                return c
    return None

def coerce_bool(series):
    if series.dtype == bool:
        return series
    mapping = {'true': True,'false': False,'yes': True,'no': False,'y': True,'n': False,'1': True,'0': False,1: True,0: False}
    return series.map(lambda x: mapping.get(str(x).strip().lower(), np.nan))

def ensure_unique_ids(df, id_col):
    if id_col is None or id_col not in df.columns:
        return df.copy(), []
    dup_mask = df[id_col].duplicated(keep='first')
    removed = df.loc[dup_mask, id_col].tolist()
    return df.loc[~dup_mask].copy(), removed

def parse_datetime_safe(s):
    try:
        return pd.to_datetime(s, errors='coerce', utc=True)
    except Exception:
        return pd.to_datetime(pd.Series(s), errors='coerce', utc=True)

# Identify key columns
aff_id_col = find_col(affiliates, ["affiliate_id", "id"])
ply_id_col = find_col(players, ["player_id", "id"])
txn_id_col = find_col(transactions, ["transaction_id", "id"])

aff_owner_player_col = find_col(affiliates, ["player_id", "owner_player_id", "affiliate_owner_player_id"])
txn_player_fk = find_col(transactions, ["player_id", "playerid"])

aff_code_col = find_col(affiliates, ["affiliate_code", "code", "ref_code", "invite_code"])
ply_redeemed_code_col = find_col(players, ["affiliate_code_redeemed", "affiliate_code_used", "affiliate_code", "ref_code", "invite_code"])

ply_kyc_bool_col = find_col(players, ["kyc_verified", "is_kyc_verified", "kyc"])
ply_kyc_date_col = find_col(players, ["kyc_verified_at", "kyc_completed_at", "kyc_date"])
txn_date_col = find_col(transactions, ["created_at", "transaction_date", "timestamp", "date"])

# Coerce types
if ply_kyc_bool_col:
    players[ply_kyc_bool_col] = coerce_bool(players[ply_kyc_bool_col])
if ply_kyc_date_col:
    players[ply_kyc_date_col] = parse_datetime_safe(players[ply_kyc_date_col])
if txn_date_col:
    transactions[txn_date_col] = parse_datetime_safe(transactions[txn_date_col])

violations = []

# Rule 5: unique IDs
affiliates, rem_aff = ensure_unique_ids(affiliates, aff_id_col)
players, rem_ply = ensure_unique_ids(players, ply_id_col)
transactions, rem_txn = ensure_unique_ids(transactions, txn_id_col)
print("Removed duplicate IDs:", {"affiliates": len(rem_aff), "players": len(rem_ply), "transactions": len(rem_txn)})

# Referential integrity: transactions.player_id in players
if txn_player_fk and ply_id_col:
    orphan_txn = transactions[~transactions[txn_player_fk].isin(players[ply_id_col])]
    print("Orphan transactions:", len(orphan_txn))
    transactions = transactions[transactions[txn_player_fk].isin(players[ply_id_col])].copy()

# Rule 2/3: affiliate code mapping
if aff_code_col is not None:
    dup_codes = affiliates[aff_code_col][affiliates[aff_code_col].duplicated(keep=False)]
    print("Duplicate affiliate codes:", dup_codes.nunique(), "rows:", dup_codes.shape[0])
    # keep first per code
    affiliates = affiliates.drop_duplicates(subset=[aff_code_col], keep='first').copy()

if ply_redeemed_code_col and aff_code_col:
    missing_code = players[players[ply_redeemed_code_col].notna() & ~players[ply_redeemed_code_col].isin(affiliates[aff_code_col])]
    print("Players with non-existent redeemed code:", len(missing_code))
    players.loc[players[ply_redeemed_code_col].isin(missing_code[ply_redeemed_code_col]), ply_redeemed_code_col] = np.nan

if aff_owner_player_col and ply_id_col:
    bad_owner = affiliates[~affiliates[aff_owner_player_col].isin(players[ply_id_col])]
    print("Affiliates with missing owner player:", len(bad_owner))
    affiliates = affiliates[affiliates[aff_owner_player_col].isin(players[ply_id_col])].copy()

# Rule 1: KYC before transactions
if txn_player_fk and ply_kyc_bool_col:
    kyc_map = players.set_index(ply_id_col)[ply_kyc_bool_col].to_dict()
    not_kyc = transactions[transactions[txn_player_fk].map(kyc_map).fillna(False) == False]
    print("Transactions by non-KYC players:", len(not_kyc))
    transactions = transactions[transactions[txn_player_fk].map(kyc_map).fillna(False)].copy()

if txn_player_fk and ply_kyc_date_col and txn_date_col:
    kyc_date_map = players.set_index(ply_id_col)[ply_kyc_date_col].to_dict()
    mask = transactions[txn_date_col] < transactions[txn_player_fk].map(kyc_date_map)
    mask = mask.fillna(False)
    before_kyc = transactions[mask]
    print("Transactions before KYC date:", len(before_kyc))
    transactions = transactions[~mask].copy()

# Save cleaned
clean_dir = base_dir / "cleaned"
clean_dir.mkdir(exist_ok=True)
affiliates.to_csv(clean_dir / "affiliates_clean.csv", index=False)
players.to_csv(clean_dir / "players_clean.csv", index=False)
transactions.to_csv(clean_dir / "transactions_clean.csv", index=False)

len(affiliates), len(players), len(transactions)

Removed duplicate IDs: {'affiliates': 0, 'players': 1, 'transactions': 0}
Orphan transactions: 0
Duplicate affiliate codes: 0 rows: 0
Transactions by non-KYC players: 3


(10, 10, 7)

## 2) Extend to 1000 Rows per Table


In [9]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from pathlib import Path
import random, string
from datetime import timedelta

# ─────────────────────────────────────────────────────────────
# 1) Paths inteligentes (detecta 'EDA_and_Generation/data' o 'data')
# ─────────────────────────────────────────────────────────────
candidates = [Path("EDA_and_Generation/data"), Path("data")]
for cand in candidates:
    if (cand / "cleaned" / "affiliates_clean.csv").exists():
        base_dir = cand
        break
else:
    raise FileNotFoundError(
        "No encontré la carpeta data. Esperaba EDA_and_Generation/data o data "
        "con los archivos cleaned/*.csv"
    )

clean_dir = base_dir / "cleaned"
ext_dir = base_dir / "extended_1000"
ext_dir.mkdir(exist_ok=True, parents=True)

# ─────────────────────────────────────────────────────────────
# 2) Lectura de datos limpios
# ─────────────────────────────────────────────────────────────
affiliates = pd.read_csv(clean_dir / "affiliates_clean.csv")
players = pd.read_csv(clean_dir / "players_clean.csv")
transactions = pd.read_csv(clean_dir / "transactions_clean.csv")

# ─────────────────────────────────────────────────────────────
# 3) Utilidades
# ─────────────────────────────────────────────────────────────
def find_col(df, candidates):
    """Busca una columna por nombre exacto (case-insensitive) o por contains."""
    cols_lc = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand in cols_lc:
            return cols_lc[cand]
    for c in df.columns:
        cl = c.lower()
        for cand in candidates:
            if cand in cl:
                return c
    return None

def random_code(n=8):
    import string, random
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=n))

def next_numeric_id(existing_ids):
    """Generador de IDs numéricos crecientes detectando máximo actual."""
    try:
        s = pd.Series(list(existing_ids)).copy()
        s_num = pd.to_numeric(s, errors='coerce')
        mx = int(np.nanmax(s_num)) if not np.all(np.isnan(s_num)) else 0
    except Exception:
        mx = 0
    cur = mx + 1
    while True:
        yield cur
        cur += 1

# Fechas siempre en UTC (aware)
NOW_UTC = pd.Timestamp.now(tz="UTC")

def _coerce_utc(series):
    return pd.to_datetime(series, errors="coerce", utc=True)

rng = np.random.default_rng(42)
target = 1000

# ─────────────────────────────────────────────────────────────
# 4) Identificación de columnas por heurística
# ─────────────────────────────────────────────────────────────
aff_id_col = find_col(affiliates, ["affiliate_id", "id"])
ply_id_col = find_col(players, ["player_id", "id"])
txn_id_col = find_col(transactions, ["transaction_id", "id"])
aff_owner_player_col = find_col(affiliates, ["player_id", "owner_player_id", "affiliate_owner_player_id"])
txn_player_fk = find_col(transactions, ["player_id", "playerid"])
aff_code_col = find_col(affiliates, ["affiliate_code", "code", "ref_code", "invite_code"])
ply_redeemed_code_col = find_col(players, ["affiliate_code_redeemed", "affiliate_code_used", "affiliate_code", "ref_code", "invite_code"])
ply_kyc_bool_col = find_col(players, ["kyc_verified", "is_kyc_verified", "kyc", "is_kyc_approved"])
ply_kyc_date_col = find_col(players, ["kyc_verified_at", "kyc_completed_at", "kyc_date"])
txn_date_col = find_col(transactions, ["created_at", "transaction_date", "timestamp", "date"])
created_col = find_col(players, ["created_at", "signup_at", "joined_at", "date_joined"])

# ─────────────────────────────────────────────────────────────
# 5) Extender affiliates
# ─────────────────────────────────────────────────────────────
aff_cols = affiliates.columns.tolist()
existing_codes = set(affiliates[aff_code_col].dropna().astype(str)) if aff_code_col else set()
owner_pool = players[ply_id_col].tolist() if (aff_owner_player_col and ply_id_col and ply_id_col in players.columns) else []
id_gen_aff = next_numeric_id(affiliates[aff_id_col] if aff_id_col else [])

n_needed = max(0, target - len(affiliates))
aff_new = []
for _ in range(n_needed):
    row = {c: np.nan for c in aff_cols}
    if aff_id_col:
        row[aff_id_col] = next(id_gen_aff)
    if aff_code_col:
        code = random_code(8)
        while code in existing_codes:
            code = random_code(8)
        row[aff_code_col] = code
        existing_codes.add(code)
    if aff_owner_player_col and owner_pool:
        row[aff_owner_player_col] = int(rng.choice(owner_pool))
    aff_new.append(row)

affiliates_ext = pd.concat([affiliates, pd.DataFrame(aff_new)], ignore_index=True)

# ─────────────────────────────────────────────────────────────
# 6) Extender players
# ─────────────────────────────────────────────────────────────
ply_cols = players.columns.tolist()
id_gen_ply = next_numeric_id(players[ply_id_col] if ply_id_col else [])
aff_codes_all = set(affiliates_ext[aff_code_col].dropna().astype(str)) if aff_code_col else set()

n_needed = max(0, target - len(players))
ply_new = []
for _ in range(n_needed):
    row = {c: np.nan for c in ply_cols}
    if ply_id_col:
        row[ply_id_col] = next(id_gen_ply)
    if created_col:
        row[created_col] = NOW_UTC - pd.Timedelta(days=int(rng.integers(0, 120))) - pd.Timedelta(minutes=int(rng.integers(0, 1440)))
    if ply_kyc_bool_col:
        row[ply_kyc_bool_col] = bool(rng.choice([True, False], p=[0.75, 0.25]))
    if ply_kyc_date_col:
        if row.get(ply_kyc_bool_col, True) is True:
            base_dt = row.get(created_col, NOW_UTC - pd.Timedelta(days=int(rng.integers(5, 90))))
            if pd.isna(base_dt):
                base_dt = NOW_UTC - pd.Timedelta(days=int(rng.integers(5, 90)))
            row[ply_kyc_date_col] = pd.to_datetime(base_dt, utc=True) + pd.Timedelta(days=int(rng.integers(0, 10)))
        else:
            row[ply_kyc_date_col] = pd.NaT
    if ply_redeemed_code_col and aff_codes_all and rng.random() < 0.6:
        row[ply_redeemed_code_col] = rng.choice(list(aff_codes_all))
    ply_new.append(row)

players_ext = pd.concat([players, pd.DataFrame(ply_new)], ignore_index=True)

# ─────────────────────────────────────────────────────────────
# 7) Normalización de vacíos/códigos
# ─────────────────────────────────────────────────────────────
def _normalize_na(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = df[c].replace(r'^\s*$', np.nan, regex=True)

def _normalize_code_series(s):
    return (
        s.astype(str)
         .str.strip()
         .str.upper()
         .replace(r'^\s*$', np.nan, regex=True)
    )

aff_cols_maybe_empty = []
if 'origin' in affiliates_ext.columns: aff_cols_maybe_empty.append('origin')
if 'redeemed_at' in affiliates_ext.columns: aff_cols_maybe_empty.append('redeemed_at')
_normalize_na(affiliates_ext, aff_cols_maybe_empty)

ply_cols_maybe_empty = []
if 'affiliate_id' in players_ext.columns: ply_cols_maybe_empty.append('affiliate_id')
_normalize_na(players_ext, ply_cols_maybe_empty)

if aff_code_col and aff_code_col in affiliates_ext.columns:
    affiliates_ext[aff_code_col] = _normalize_code_series(affiliates_ext[aff_code_col])
if ply_redeemed_code_col and ply_redeemed_code_col in players_ext.columns:
    players_ext[ply_redeemed_code_col] = _normalize_code_series(players_ext[ply_redeemed_code_col])

# ─────────────────────────────────────────────────────────────
# 8) Si players no tiene código redimido, crear temporalmente y poblar (~60%)
# ─────────────────────────────────────────────────────────────
created_temp_redeemed_col = False
if not ply_redeemed_code_col:
    ply_redeemed_code_col = "affiliate_code_redeemed"
    if ply_redeemed_code_col not in players_ext.columns:
        players_ext[ply_redeemed_code_col] = np.nan
        created_temp_redeemed_col = True

valid_aff_codes = []
if aff_code_col and aff_code_col in affiliates_ext.columns:
    valid_aff_codes = (
        affiliates_ext[aff_code_col]
        .astype(str).str.strip().str.upper()
        .replace(r'^\s*$', np.nan, regex=True)
        .dropna().unique().tolist()
    )

if valid_aff_codes:
    mask_no_code = players_ext[ply_redeemed_code_col].replace(r'^\s*$', np.nan, regex=True).isna()
    if mask_no_code.any():
        assign_mask = mask_no_code & (rng.random(len(players_ext)) < 0.60)
        n_assign = int(assign_mask.sum())
        if n_assign > 0:
            players_ext.loc[assign_mask, ply_redeemed_code_col] = rng.choice(valid_aff_codes, size=n_assign)

# normaliza de nuevo el código redimido tras poblar
players_ext[ply_redeemed_code_col] = (
    players_ext[ply_redeemed_code_col]
    .astype(str).str.strip().str.upper()
    .replace(r'^\s*$', np.nan, regex=True)
)

# ─────────────────────────────────────────────────────────────
# 9) Columnas extra (origin, redeemed_at, affiliate_id, country_code, updated_at)
# ─────────────────────────────────────────────────────────────
# affiliates.origin (solo completa faltantes)
if "origin" not in affiliates_ext.columns:
    affiliates_ext["origin"] = np.nan
origin_pool = ["web", "ios", "android", "partner", "campaign", "YouTube", "Discord", "X"]
mask_origin = affiliates_ext["origin"].isna()
if mask_origin.any():
    affiliates_ext.loc[mask_origin, "origin"] = rng.choice(origin_pool, size=int(mask_origin.sum()))

# affiliates.redeemed_at (best-of created_at/updated_at/kyc_date) + jitter
if "redeemed_at" not in affiliates_ext.columns:
    affiliates_ext["redeemed_at"] = pd.NaT
else:
    affiliates_ext["redeemed_at"] = affiliates_ext["redeemed_at"].replace(r'^\s*$', np.nan, regex=True)
    affiliates_ext["redeemed_at"] = _coerce_utc(affiliates_ext["redeemed_at"])

date_candidates = []
if created_col and created_col in players_ext.columns:
    date_candidates.append(created_col)
if "updated_at" in players_ext.columns:
    date_candidates.append("updated_at")
if ply_kyc_date_col and ply_kyc_date_col in players_ext.columns:
    date_candidates.append(ply_kyc_date_col)

# Columna temporal en UTC para evitar FutureWarning
tmp_date_col = "__best_player_date__"
players_ext[tmp_date_col] = pd.Series(pd.NaT, index=players_ext.index, dtype="datetime64[ns, UTC]")

if aff_code_col and ply_redeemed_code_col and date_candidates:
    for c in date_candidates:
        if c in players_ext.columns:
            mask_tmp = players_ext[tmp_date_col].isna()
            if mask_tmp.any():
                players_ext.loc[mask_tmp, tmp_date_col] = _coerce_utc(players_ext.loc[mask_tmp, c])

    used = players_ext.dropna(subset=[ply_redeemed_code_col])
    if not used.empty:
        used_codes_norm = used[ply_redeemed_code_col].astype(str).str.strip().str.upper()
        code_to_first = (
            pd.DataFrame({
                "code_norm": used_codes_norm,
                "first_dt": _coerce_utc(used[tmp_date_col])
            })
            .dropna(subset=["first_dt"])
            .groupby("code_norm", as_index=True)["first_dt"]
            .min()
        )

        aff_code_norm = affiliates_ext[aff_code_col].astype(str).str.strip().str.upper()
        mask_red = affiliates_ext["redeemed_at"].isna()
        if mask_red.any():
            mapped = aff_code_norm.map(code_to_first)
            mapped_dt = _coerce_utc(mapped)
            affiliates_ext.loc[mask_red, "redeemed_at"] = mapped_dt.loc[mask_red]

            # jitter a los recién rellenados
            mask_after = affiliates_ext["redeemed_at"].notna() & mask_red
            if mask_after.any():
                jitter = pd.to_timedelta(rng.integers(0, 60*24, size=int(mask_after.sum())), unit="m")
                affiliates_ext.loc[mask_after, "redeemed_at"] = affiliates_ext.loc[mask_after, "redeemed_at"] + jitter

# players.affiliate_id (map por código → affiliate_id; completa faltantes)
if aff_id_col and aff_code_col and ply_redeemed_code_col:
    code_to_affid = affiliates_ext.copy()
    code_to_affid[aff_code_col] = code_to_affid[aff_code_col].astype(str).str.strip().str.upper()
    code_to_affid = code_to_affid.set_index(aff_code_col)[aff_id_col]
    try:
        code_to_affid = pd.to_numeric(code_to_affid, errors="coerce").astype("Int64")
    except Exception:
        pass
    code_to_affid = code_to_affid.to_dict()

    if "affiliate_id" not in players_ext.columns:
        players_ext["affiliate_id"] = np.nan

    mask_affid = players_ext["affiliate_id"].replace(r'^\s*$', np.nan, regex=True).isna()
    if mask_affid.any():
        players_ext.loc[mask_affid, "affiliate_id"] = players_ext.loc[mask_affid, ply_redeemed_code_col].map(code_to_affid)

    try:
        players_ext["affiliate_id"] = pd.to_numeric(players_ext["affiliate_id"], errors="coerce").astype("Int64")
    except Exception:
        pass
else:
    if "affiliate_id" not in players_ext.columns:
        players_ext["affiliate_id"] = np.nan

# players.country_code (no pisa existentes)
country_pool = ["CO","US","MX","BR","AR","CL","PE","ES","GB","DE","FR","CA","IN","PH","AU"]
if "country_code" not in players_ext.columns:
    players_ext["country_code"] = np.nan
mask_cc = players_ext["country_code"].isna()
players_ext.loc[mask_cc, "country_code"] = rng.choice(country_pool, size=int(mask_cc.sum()))

# players.updated_at ≥ created_at (o base plausible si no hay created_at)
if "updated_at" not in players_ext.columns:
    players_ext["updated_at"] = pd.NaT

def _rand_after(dt):
    if pd.notna(dt):
        base = pd.to_datetime(dt, utc=True)  # asegura aware
    else:
        base = pd.Timestamp.now(tz="UTC") - pd.Timedelta(days=int(rng.integers(0, 120)))
    return base + pd.Timedelta(days=int(rng.integers(0, 30))) + pd.Timedelta(minutes=int(rng.integers(0, 1440)))

mask_upd = players_ext["updated_at"].isna()
if created_col and created_col in players_ext.columns:
    players_ext.loc[mask_upd, "updated_at"] = [
        _rand_after(dt) for dt in players_ext.loc[mask_upd, created_col]
    ]
else:
    players_ext.loc[mask_upd, "updated_at"] = [
        _rand_after(pd.NaT) for _ in range(int(mask_upd.sum()))
    ]

# ─────────────────────────────────────────────────────────────
# 10) Extender transactions (sólo para jugadores KYC)
# ─────────────────────────────────────────────────────────────
txn_cols = transactions.columns.tolist()
id_gen_txn = next_numeric_id(transactions[txn_id_col] if txn_id_col else [])

# asegura booleanos/fechas
if ply_kyc_bool_col and ply_kyc_bool_col in players_ext.columns:
    players_ext[ply_kyc_bool_col] = players_ext[ply_kyc_bool_col].astype(bool)

if created_col and created_col in players_ext.columns:
    players_ext[created_col] = _coerce_utc(players_ext[created_col])
if "updated_at" in players_ext.columns:
    players_ext["updated_at"] = _coerce_utc(players_ext["updated_at"])
if ply_kyc_date_col and ply_kyc_date_col in players_ext.columns:
    players_ext[ply_kyc_date_col] = _coerce_utc(players_ext[ply_kyc_date_col])
if "redeemed_at" in affiliates_ext.columns:
    affiliates_ext["redeemed_at"] = _coerce_utc(affiliates_ext["redeemed_at"])

kyc_map = players_ext.set_index(ply_id_col)[ply_kyc_bool_col].to_dict() if ply_kyc_bool_col else {}
kyc_date_map = players_ext.set_index(ply_id_col)[ply_kyc_date_col].to_dict() if ply_kyc_date_col else {}
player_pool = players_ext[ply_id_col].dropna().astype(int).tolist() if ply_id_col else []
kyc_players = [pid for pid in player_pool if kyc_map.get(pid, False) is True]

n_needed = max(0, target - len(transactions))
txn_new = []
for _ in range(n_needed):
    if not kyc_players:
        break
    pid = int(rng.choice(kyc_players))
    row = {c: np.nan for c in txn_cols}
    if txn_id_col:
        row[txn_id_col] = next(id_gen_txn)
    if txn_player_fk:
        row[txn_player_fk] = pid
    if txn_date_col:
        base_dt = kyc_date_map.get(pid, pd.Timestamp.now(tz="UTC") - pd.Timedelta(days=30))
        if pd.isna(base_dt):
            base_dt = pd.Timestamp.now(tz="UTC") - pd.Timedelta(days=30)
        row[txn_date_col] = pd.to_datetime(base_dt, utc=True) + pd.Timedelta(days=int(rng.integers(0, 60))) + pd.Timedelta(minutes=int(rng.integers(0, 1440)))
    # Completar columnas por tipo tomando como referencia el dtype de 'transactions'
    for c in transactions.columns:
        if pd.notna(row.get(c)):
            continue
        cl = c.lower()
        if transactions[c].dtype == object:
            if "currency" in cl:
                row[c] = rng.choice(["USD","EUR","COP","BRL"])
            elif "type" in cl:
                row[c] = rng.choice(["deposit","withdrawal","bet","win"])
            else:
                row[c] = None
        elif np.issubdtype(transactions[c].dtype, np.number):
            if "amount" in cl or "value" in cl:
                row[c] = float(int(rng.integers(1, 5000)))
            else:
                row[c] = float(int(rng.integers(0, 1000)))
        elif np.issubdtype(transactions[c].dtype, np.datetime64):
            # ya seteado txn_date_col si aplica
            pass
    txn_new.append(row)

transactions_ext = pd.concat([transactions, pd.DataFrame(txn_new)], ignore_index=True)

# coerción de fecha en transacciones (seguridad)
if txn_date_col and txn_date_col in transactions_ext.columns:
    transactions_ext[txn_date_col] = _coerce_utc(transactions_ext[txn_date_col])

# ─────────────────────────────────────────────────────────────
# 11) Tipos consistentes en IDs (suave)
# ─────────────────────────────────────────────────────────────
for df in [affiliates_ext, players_ext, transactions_ext]:
    for col in [aff_id_col, ply_id_col, txn_id_col, txn_player_fk, "affiliate_id"]:
        if col and col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
            except Exception:
                pass

# ─────────────────────────────────────────────────────────────
# 12) Limpieza final de columnas auxiliares
# ─────────────────────────────────────────────────────────────
if "__best_player_date__" in players_ext.columns:
    players_ext.drop(columns=["__best_player_date__"], inplace=True)
# elimina affiliate_code_redeemed solo si lo creamos temporalmente
if 'created_temp_redeemed_col' in locals() and created_temp_redeemed_col and "affiliate_code_redeemed" in players_ext.columns:
    players_ext.drop(columns=["affiliate_code_redeemed"], inplace=True)

# ─────────────────────────────────────────────────────────────
# 13) Reglas de negocio (validaciones)
# ─────────────────────────────────────────────────────────────
# 1. Nadie transacciona antes de estar KYC (y fecha posterior al KYC)
if ply_kyc_bool_col and txn_player_fk and txn_date_col and ply_kyc_date_col:
    merged = transactions_ext.merge(
        players_ext[[ply_id_col, ply_kyc_bool_col, ply_kyc_date_col]],
        left_on=txn_player_fk, right_on=ply_id_col, how="left"
    )
    bad_kyc = merged[(merged[ply_kyc_bool_col] != True) | (merged[txn_date_col] < merged[ply_kyc_date_col])]
    assert bad_kyc.empty, f"Hay {len(bad_kyc)} transacciones violando KYC (no verificado o fecha previa al KYC)."

# 2. affiliate_id ↔ código redimido coherente (si existen ambas)
if aff_id_col and aff_code_col and "affiliate_id" in players_ext.columns and ply_redeemed_code_col in players_ext.columns:
    code_by_affid = affiliates_ext[[aff_id_col, aff_code_col]].dropna().copy()
    code_by_affid[aff_code_col] = code_by_affid[aff_code_col].astype(str).str.strip().str.upper()
    code_by_affid = code_by_affid.drop_duplicates(subset=[aff_id_col])

    chk = players_ext.merge(code_by_affid, left_on="affiliate_id", right_on=aff_id_col, how="left", suffixes=("","__aff"))
    mism = chk[
        chk["affiliate_id"].notna() &
        (ply_redeemed_code_col in chk.columns) &
        chk[ply_redeemed_code_col].notna() &
        (chk[ply_redeemed_code_col].astype(str).str.strip().str.upper() != chk[aff_code_col].astype(str).str.strip().str.upper())
    ]
    assert mism.empty, f"{len(mism)} jugadores con affiliate_id cuyo código no coincide con el código del afiliado."

# 3. IDs únicos
if ply_id_col:
    assert players_ext[ply_id_col].nunique() == len(players_ext), "player_id no es único"
if aff_id_col:
    assert affiliates_ext[aff_id_col].nunique() == len(affiliates_ext), "affiliate_id no es único"
if txn_id_col:
    assert transactions_ext[txn_id_col].nunique() == len(transactions_ext), "transaction_id no es único"

# 4. Conteos objetivo
assert len(affiliates_ext)   >= 1000, f"Affiliates < 1000 (={len(affiliates_ext)})"
assert len(players_ext)      >= 1000, f"Players < 1000 (={len(players_ext)})"
assert len(transactions_ext) >= 1000, f"Transactions < 1000 (={len(transactions_ext)})"

# ─────────────────────────────────────────────────────────────
# 14) Guardar
# ─────────────────────────────────────────────────────────────
affiliates_ext.to_csv(ext_dir / "affiliates_1000.csv", index=False)
players_ext.to_csv(ext_dir / "players_1000.csv", index=False)
transactions_ext.to_csv(ext_dir / "transactions_1000.csv", index=False)

print(
    "OK ✅ Datos extendidos y validados",
    f"\nAffiliates:   {len(affiliates_ext)}",
    f"\nPlayers:      {len(players_ext)}",
    f"\nTransactions: {len(transactions_ext)}",
    f"\nArchivos en:  {ext_dir.resolve()}",
)


OK ✅ Datos extendidos y validados 
Affiliates:   1000 
Players:      1000 
Transactions: 1000 
Archivos en:  /Users/jfts/Documents/ancientg/EDA_and_Generation/data/extended_1000


/var/folders/18/8hb3mrl17z58hycjwy8kspt00000gn/T/ipykernel_36909/3945220213.py:202: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['DK0TB2VK' 'IRY7BF52' '2O0J9L8T' 'VQE7YAS4' 'MU3O32RE' 'JA4NQGHR'
 'CP9CTRT6' '5L6CQN0W' 'QRGR0HH4' 'DCT529QN' '0XMREARM' 'SF06Y4ZP'
 'RE5F8FNP' 'ENJ9VY7W' 'H4J84NET' 'UG3QYN82' 'W5KS3BDU' 'JWMYC19H'
 'MRUG72XF' '4D0JIICP' 'K34GKW0G' '04HFVALU' 'NCSZJNIS' '9JNWHVOL'
 'I8CX8ZJP' 'N3UBFXEQ' 'GZ6IAUJ4' 'UG3QYN82' 'X0RRUPNK' 'V2G5DM56'
 'AJ32LJI8' 'J5TCYZHA' 'CWT9PS86' 'E50HX378' '8LXY4SCK' 'YAFGLUCL'
 'I6J1YV31' 'R6QCMFXW' 'BS49TODF' '0EVN0F7O' 'H3LTCM2D' '2O0J9L8T'
 '6PH42POM' 'KJX6JMFE' 'Z4J7P31L' '9Y41OXNO' 'XDHSUUV6' '0XMREARM'
 'WYYEGIDY' 'STEHZLNU' '7QW30S8L' 'NM6LYS2G' 'US1V5WQI' 'KVQ5DFL1'
 'FCPV0YV5' 'YN29UVDG' '14LEBD9R' 'ZQ7RU2NB' 'OE8YBWJ5' 'YJ2IPGDV'
 'R3MR875M' 'SX10S84U' '89Z14T3W' 'QYRXALSO' '2026PHU1' 'M0CEM46X'
 'GVBH1DC2' '109QZ4KF' '558LDYCA' 'VQE7YAS4' 'ECLY